<a href="https://colab.research.google.com/github/WaseemMohammed-20/AI_Emotion_Detector/blob/main/AI_Emotion_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [2]:
data = {
    "text": [
        "I am feeling very happy today",
        "This is the worst day ever",
        "I feel sad and depressed",
        "I am extremely excited about my results",
        "I am angry about what happened",
        "I feel normal today",
        "This makes me furious",
        "I am joyful and smiling",
        "I feel lonely",
        "Nothing special, just another day"
    ],
    "emotion": [
        "Happy", "Angry", "Sad", "Happy", "Angry",
        "Neutral", "Angry", "Happy", "Sad", "Neutral"
    ]
}

df = pd.DataFrame(data)
df


,text,emotion
0,I am feeling very happy today,Happy
1,This is the worst day ever,Angry
2,I feel sad and depressed,Sad
3,I am extremely excited about my results,Happy
4,I am angry about what happened,Angry
5,I feel normal today,Neutral
6,This makes me furious,Angry
7,I am joyful and smiling,Happy
8,I feel lonely,Sad
9,"Nothing special, just another day",Neutral


In [3]:
X = df['text']     # Input (text)
y = df['emotion']  # Output (emotion)


In [4]:
vectorizer = TfidfVectorizer(stop_words='english')

X_vectors = vectorizer.fit_transform(X)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y, test_size=0.2, random_state=42
)


In [6]:
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [7]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.0


In [8]:
def predict_emotion(text):
    text_vector = vectorizer.transform([text])
    prediction = model.predict(text_vector)
    return prediction[0]


In [9]:
predict_emotion("I failed my exam and feel terrible")


'Neutral'

In [10]:
predict_emotion("I got selected for my dream job")


'Happy'

In [11]:
predict_emotion("Today feels normal")


'Neutral'

# New section

In [15]:
import pandas as pd
import numpy as np
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [16]:
data = {
    "text": [
        "I am very happy today",
        "This is the worst day ever",
        "I feel sad and lonely",
        "I am extremely excited",
        "I am angry about this",
        "Feeling normal today",
        "I am joyful",
        "I feel depressed",
        "This makes me furious",
        "Nothing special today"
    ],
    "emotion": [
        "Happy", "Angry", "Sad", "Happy", "Angry",
        "Neutral", "Happy", "Sad", "Angry", "Neutral"
    ]
}

df = pd.DataFrame(data)


In [17]:
X = df["text"]
y = df["emotion"]


In [18]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_vectors = vectorizer.fit_transform(X)


In [19]:
model = LogisticRegression(max_iter=1000)
model.fit(X_vectors, y)


LogisticRegression(max_iter=1000)

In [20]:
joblib.dump(model, "emotion_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [21]:
model = joblib.load("emotion_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")


In [22]:
emoji_map = {
    "Happy": "😊",
    "Sad": "😢",
    "Angry": "😡",
    "Neutral": "😐"
}


In [23]:
def predict_emotion(text):
    if text.strip() == "":
        return "Please enter valid text ❌"

    vector = vectorizer.transform([text])
    prediction = model.predict(vector)[0]
    probability = model.predict_proba(vector).max() * 100
    emoji = emoji_map.get(prediction, "")

    return f"Emotion: {prediction} {emoji} | Confidence: {probability:.2f}%"


In [24]:
predict_emotion("I got selected for my dream job")


'Emotion: Angry 😡 | Confidence: 30.98%'

In [25]:
predict_emotion("I failed my exam and feel terrible")


'Emotion: Sad 😢 | Confidence: 33.59%'

In [26]:
predict_emotion("Today feels normal")


'Emotion: Neutral 😐 | Confidence: 31.15%'

# New section

In [28]:
!pip install googletrans==4.0.0-rc1


In [29]:
from googletrans import Translator


In [30]:
translator = Translator()


In [31]:
def translate_to_english(text):
    detected = translator.detect(text)
    lang = detected.lang

    if lang != "en":
        translated = translator.translate(text, src=lang, dest="en")
        return translated.text, lang
    else:
        return text, "en"


In [32]:
def predict_emotion_multilingual(text):
    if text.strip() == "":
        return "Please enter valid text ❌"


    translated_text, lang = translate_to_english(text)

    vector = vectorizer.transform([translated_text])


    prediction = model.predict(vector)[0]
    confidence = model.predict_proba(vector).max() * 100
    emoji = emoji_map.get(prediction, "")

    language_map = {
        "en": "English",
        "hi": "Hindi",
        "te": "Telugu"
    }

    detected_language = language_map.get(lang, lang)

    return (
        f"Detected Language: {detected_language}\n"
        f"Translated Text: {translated_text}\n"
        f"Emotion: {prediction} {emoji}\n"
        f"Confidence: {confidence:.2f}%"
    )


In [33]:
predict_emotion_multilingual("I got selected for my dream job")


'Detected Language: English\nTranslated Text: I got selected for my dream job\nEmotion: Angry 😡\nConfidence: 30.98%'

In [34]:
predict_emotion_multilingual("मैं बहुत खुश हूँ")


'Detected Language: Hindi\nTranslated Text: I am very happy\nEmotion: Happy 😊\nConfidence: 40.95%'

In [35]:
predict_emotion_multilingual("నేను చాలా బాధగా ఉన్నాను")


'Detected Language: Telugu\nTranslated Text: I am very sad\nEmotion: Angry 😡\nConfidence: 28.21%'